In [29]:
%matplotlib inline
import sys
sys.path.append('../../stlcg_karen/src')
sys.path.append('../../stlcg_karen/src')
import stlcg
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from torch.utils.data import Dataset, DataLoader

import numpy as np
import os
import pickle
import torch
import pickle
import scipy.io as sio

from torch_interpolations.torch_interpolations.multilinear import RegularGridInterpolator

In [200]:
value = sio.loadmat('../hji/data/value.mat');
deriv_value = sio.loadmat('../hji/data/deriv_value.mat');
V = value['data'];
dV = [deriv_value['derivC'][i][0] for i in range(4)];
g = sio.loadmat('../hji/data/grid.mat')['grid'];


In [296]:

values = torch.tensor(V[:,:,:,:,-1]).float()
points = [torch.from_numpy(g[i][0].flatten()).float() for i in range(4)]
value_interp = RegularGridInterpolator(points, values)
deriv_interp = [RegularGridInterpolator(points, torch.tensor(dV[i][:,:,:,:,-1]).float()) for i in range(4)]

In [255]:
[p[:3] for p in points]

[tensor([-5.0000, -4.5000, -4.0000]),
 tensor([0.0000, 0.3000, 0.6000]),
 tensor([-3.1416, -2.9883, -2.8351]),
 tensor([0.0000, 0.2500, 0.5000])]

In [298]:
class HJIValueFunction(torch.autograd.Function):
        
    @staticmethod
    def forward(ctx, input):
        """
        [bs, x_dim]
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return value_interp(input.split(1, dim=1))

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        points = input.split(1, dim=1)
        gr = torch.zeros_like(input)
        return  torch.cat([deriv_interp[i](points) for i in range(4)], 1) * grad_output

In [299]:
vf = HJIValueFunction.apply


In [312]:
inputs = torch.stack([p[:3] for p in points], 1).float().requires_grad_(True)
inputs

tensor([[-5.0000,  0.0000, -3.1416,  0.0000],
        [-4.5000,  0.3000, -2.9883,  0.2500],
        [-4.0000,  0.6000, -2.8351,  0.5000]], requires_grad=True)

In [322]:
loss = vf(inputs[1:2,:]).squeeze()

In [323]:
loss.backward()

In [324]:
inputs.grad

tensor([[-0.4053, -0.5063,  9.4112, -1.3243],
        [-0.2907, -0.4539,  2.7025, -0.4282],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

In [321]:
torch.stack([deriv_interp[i]([p[:3] for p in points]) for i in range(4)], 1)

tensor([[-0.4053, -0.5063,  9.4112, -1.3243],
        [-0.2907, -0.4539,  2.7025, -0.4282],
        [-0.1980, -0.4046, -0.4952, -0.0190]])

In [317]:
inputs

tensor([[-5.0000,  0.0000, -3.1416,  0.0000],
        [-4.5000,  0.3000, -2.9883,  0.2500],
        [-4.0000,  0.6000, -2.8351,  0.5000]], requires_grad=True)

In [18]:
data = np.load("../hji/data/expert_traj.npy")


In [22]:
class ExpertDemoDataset(torch.utils.data.Dataset):

    def __init__(self, npy_file):
        
        # [t, x, y, psi, V]
        self.data = np.load(npy_file)


    def __len__(self):
        return data.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        time = self.data[idx, 0]
        state = self.data[idx, 1:5]
        control = self.data[idx, 5:7]
        return {'state': state, 'control': control, 'time': time}


In [51]:
input_size = 4
hidden_size = 8
num_layers = 1
ctrl_dim = 2
lstm = torch.nn.LSTM(input_size, hidden_size, num_layers)
proj = torch.nn.Linear(hidden_size, ctrl_dim)

In [52]:
inputs = torch.tensor(data[:,1:5]).float().unsqueeze(1)

In [60]:
o, (h,c) = lstm(inputs)

In [66]:
proj(h).shape

torch.Size([1, 1, 2])

In [57]:
?lstm

In [75]:
def dynamics(x_prev, u):
    return 2 * x_prev

In [77]:
n = 10
x_partial = inputs[:10, :,:]
o, (h, c) = lstm(x_partial)    # h is the last hidden state/last output
u = proj(h)    # [1, bs, ctrl_dim]
x_future = []
x_prev = x_partial[-1:, :,:]

for i in range(n):
    x_next = dynamics(x_prev, u)
    x_future.append(x_next)
    u = lstm(x_next, (h,c))
    x_prev = x_next
return

In [84]:
loss = torch.nn.MSELoss()

In [88]:
import scipy.io as sio


In [27]:
value = sio.loadmat('../hji/data/value.mat');
deriv_value = sio.loadmat('../hji/data/deriv_value.mat');
V = value['data'];
dV = [deriv_value['derivC'][i][0] for i in range(4)];

NameError: name 'sio' is not defined

In [134]:
V_tensor = torch.tensor(V[:3,:3,0,0,-1]).float()
V_tensor.shape

torch.Size([3, 3])

In [144]:
m = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

In [153]:
grid = torch.arange(3, dtype=torch.float32).view(1, 1, 3, 1)
torch.

In [159]:
grid.repeat([1,1,1,3])

tensor([[[[0., 0., 0.],
          [1., 1., 1.],
          [2., 2., 2.]]]])

In [145]:
V_tensor.unsqueeze(0).unsqueeze(1)

tensor([[[[5.0005, 4.8486, 4.6865],
          [4.7978, 4.6409, 4.4837],
          [4.6112, 4.4454, 4.2903]]]])

In [146]:
m(V_tensor.unsqueeze(0).unsqueeze(1))

tensor([[[[5.0005, 4.9397, 4.8790, 4.8162, 4.7513, 4.6865],
          [4.9194, 4.8579, 4.7963, 4.7335, 4.6694, 4.6054],
          [4.8384, 4.7760, 4.7136, 4.6508, 4.5875, 4.5243],
          [4.7605, 4.6970, 4.6335, 4.5705, 4.5077, 4.4450],
          [4.6858, 4.6209, 4.5561, 4.4924, 4.4300, 4.3676],
          [4.6112, 4.5449, 4.4786, 4.4144, 4.3523, 4.2903]]]])

In [149]:
V_tensor[0,:].numpy()

array([5.000489, 4.848586, 4.686505], dtype=float32)

In [87]:
class STLPolicy(torch.nn.Module):

    def __init__(self, state_dim, ctrl_dim, hidden_dim, num_layers=1):
        super(STLPolicy, self).__init__()
        self.lstm = torch.nn.LSTM(state_dim, hidden_dim, num_layers)
        self.proj = torch.nn.Linear(hidden_dim, ctrl_dim)
        self.L2loss = torch.nn.MSELoss()

    
    def forward(self, x):        
        # x is [time_dim, bs, state_dim]
        o, _ = self.lstm(x)    # [time_dim, bs, hidden_dim] , bs = 1 for a single expert trajectory.
        u = self.proj(o)    # [time_dim, bs, ctrl_dim]
        return o, u
            
    
    def propagate_n(self, n, x_partial, dynamics):
        '''
        n is the number of time steps to propagate forward
        x_partial is the input trajectory [time_dim, bs, state_dim]
        dynamics is a function that takes in x and u and gives the next state
        '''
        o, (h, c) = self.lstm(x_partial)    # h is the last hidden state/last output
        u = self.proj(h)    # [1, bs, ctrl_dim]
        x_future = []
        x_prev = x_partial[-1:, :,:]

        for i in range(n):
            x_next = dynamics(x_prev, u)
            x_future.append(x_next)
            u = self.lstm(x_next, (h,c))
            x_prev = x_next
                
        return torch.cat(x_future, 0)
        
        
    def control_loss(self, x_partial, u_true):
        o, u = self.forward(x_partial)
        return self.L2loss(u, u_true)
    
    def join_partial_future_signal(self, x_partial, x_future):
        return torch.cat([x_partial, x_future], 0)
    
    def STL_loss_n(self, n, x_partial, formula, formula_input_func, dynamics, **kwargs):

        x_future = self.propagate_n(n, x_partial, dynamics)
        x_complete = self.join_partial_future_signal(x_partial, x_future)
        return torch.relu(-formula.robustness(formula_input_func(x_complete), **kwargs)).mean()
    
    def STL_loss(self, x, formula, formula_input_func, **kwargs):
        return torch.relu(-formula.robustness(formula_input_func(x), **kwargs)).mean()
    
#     def 
#     def HJI_loss(self):
        

In [291]:
qwe = "popo"

In [293]:
vf = HJIValueFunction().apply

In [294]:
vf(inputs)

popo


TypeError: HJIValueFunctionBackward.forward: expected Variable (got int) for return value 0

In [ ]:
class MyReLU(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

In [1]:
def V(x, y):
    return x + y

In [4]:
corners = np.array([[0, 1, 1, 0],[0, 0, 1, 1]])

In [11]:
x0 = 0.3
y0 = 0.7


In [46]:
def interp1D(x0, xs, vs):
    gamma = (x0 - xs[:,:1])/(xs[:,1:] - xs[:,:1])
    return (1 - gamma) * vs[:,:1] + gamma * vs[:,1:]

In [32]:
x0 = 0.3 * torch.ones([10, 1])
xs = torch.zeros([10, 2])
xs[:,1] = 1


In [44]:
interp1D(x0, xys, vs)

tensor([[0.3000],
        [0.3000],
        [0.3000],
        [0.3000],
        [0.3000],
        [0.3000],
        [0.3000],
        [0.3000],
        [0.3000],
        [0.3000]])

In [ ]:
# order of corners [bl, br, tr tl]
# xy0 [bs, 2]
# xys [bs, 4, 2]
# vs [bs, 4]

torch.Size([10, 4, 2])

In [60]:
bs = 10
xy0 = torch.zeros([bs, 2])
xy0[:,0] = 0.3
xy0[:,1] = 0.7
xys = torch.cat([torch.tensor([0, 1, 1, 0]).float().unsqueeze(0).unsqueeze(-1).repeat([bs, 1, 1]), torch.tensor([0, 0, 1, 1]).float().unsqueeze(0).unsqueeze(-1).repeat([bs, 1, 1])], dim=-1)
vs = torch.zeros([bs, 4])
vs[:,1] = 1
vs[:,2] = 2
vs[:,3] = 1

In [67]:
gamma = (xy0 - xys[:,0,:]) / (xys[:,2,:] - xys[:,0,:])    # [bs, 2]

In [69]:
(1 - gamma)

tensor([[0.7000, 0.3000],
        [0.7000, 0.3000],
        [0.7000, 0.3000],
        [0.7000, 0.3000],
        [0.7000, 0.3000],
        [0.7000, 0.3000],
        [0.7000, 0.3000],
        [0.7000, 0.3000],
        [0.7000, 0.3000],
        [0.7000, 0.3000]])

In [13]:
def interp2D(xy0, xys, vs):
    

1.2999999999999998

In [7]:
V(corners[0,:], corners[1,:])

array([0, 1, 2, 1])

In [2]:
print(torch.__version__)

1.3.0
